In this tutorial we will see what it takes to render a textured quad using the pyGanadalf API.
----------------------------------------------------------------------------------------------

Before doing anything else we need to import the nessasary packages

In [1]:
from pyGandalf.core.application import Application
from pyGandalf.core.opengl_window import OpenGLWindow
from pyGandalf.systems.transform_system import TransformSystem
from pyGandalf.systems.opengl_rendering_system import OpenGLStaticMeshRenderingSystem

from pyGandalf.renderer.opengl_renderer import OpenGLRenderer

from pyGandalf.scene.scene import Scene
from pyGandalf.scene.components import *
from pyGandalf.scene.scene_manager import SceneManager

from pyGandalf.utilities.opengl_material_lib import OpenGLMaterialLib, MaterialData
from pyGandalf.utilities.opengl_texture_lib import OpenGLTextureLib, TextureData, TextureDescriptor
from pyGandalf.utilities.opengl_shader_lib import OpenGLShaderLib

from pyGandalf.utilities.definitions import SHADERS_PATH, TEXTURES_PATH

import numpy as np
import glm

First, we create a new application. To achieve this, we need to specify two main things:
- The window
  - We create an `OpenGLWindow` and we specify the name, width, height and vertical sync mode accordingly.
- The renderer
  - We specify that we will use the `OpenGLRenderer`.

In [2]:
Application().create(OpenGLWindow('Textured Quad', 1280, 720, True), OpenGLRenderer)

Then, we build a texture, lets take a closer look at how this can be achieved.
- 1. From the singleton class `OpenGLTextureLib` we call the `build` method to build our shader.
- 2. We give our texture a name, in this case we chose: "uoc_logo".
- 3. We provide the build function with the data and descriptor of the texture
  - TextureData: Holds the data of the texture. You can either give a path (or list of paths if cubemap) or the byte data to use when creating the texture and the width and height.
  - TextureDescriptor: The description of the texture, which consists of various options and flags. Here, we specify that we want to flip the texture.

In [3]:
OpenGLTextureLib().build('uoc_logo', TextureData(path=TEXTURES_PATH / 'uoc_logo.png'), TextureDescriptor(flip=True))

0

Then we build a shader, lets take a closer look at how this can be achieved.
- 1. From the singleton class `OpenGLShaderLib` we call the `build` method to build our shader.
- 2. We give our shader a name, in this case we chose: "unlit_textured".
- 3. We provide the build function with the path to the source code of the shader stages that we want our shader to utilize, in this case we use a vertext and fragment shader.

In [4]:
OpenGLShaderLib().build('unlit_textured', SHADERS_PATH / 'opengl' / 'unlit_textured.vs', SHADERS_PATH / 'opengl' / 'unlit_textured.fs')

3

Then we build a material, lets take a closer look at how this can be achieved.
- 1. From the singleton class `OpenGLMaterialLib` we call the `build` method to build our material.
- 2. We give the material a name, in this case we chose: "M_Unlit".
- 3. We give specify the desired data of the material through the `MaterialData` class.
  - As the first argument, we give them shader name that we want to use, we give the name of the shader we created above, meaning: "unlit_textured".
  - The second argument is used to define which textures the material we use, so we add the name of the texture we created above i.e.: "uoc_logo".

In [5]:
OpenGLMaterialLib().build('M_UnlitTextured', MaterialData('unlit_textured', ['uoc_logo']))

We create a new scene by instatiating an object of the class `Scene`, we give it the name "Textured Quad".

In [6]:
scene = Scene('Textured Quad')

Then, we enroll a new entity to the scene by calling the `enroll_entity` method. This entity will represent our quad.

In [7]:
quad = scene.enroll_entity()

Now, its time to add components to our newly created entity, i.e.: `quad`.

First, we using our `scene` and its `add_component` method, we add to the `quad` entity, a `TransformComponent`.

We when are creating the `TransformComponent`, we specify the translation, rotation and scale vectors.

In [8]:
scene.add_component(quad, TransformComponent(glm.vec3(0, 0, 0), glm.vec3(0, 0, 0), glm.vec3(1, 1, 1)))

Then, we want to add a `StaticMeshComponent` component to the `triangle` entity.

But first we need to specify the vertex data of the quad, we can do that by using numpy arrays like this:

In [9]:
vertices = np.array([
    [-0.5, -0.5, 0.0], # 0 - Bottom left corner
    [ 0.5, -0.5, 0.0], # 1 - Bottom right corner
    [ 0.5,  0.5, 0.0], # 2 - Top right corner
    [ 0.5,  0.5, 0.0], # 2 - Top right corner
    [-0.5,  0.5, 0.0], # 3 - Top left corner
    [-0.5, -0.5, 0.0]  # 0 - Bottom left corner
], dtype=np.float32)

# Texture coordinates of the quad
texture_coords = np.array([
    [0.0, 1.0], # 0
    [1.0, 1.0], # 1
    [1.0, 0.0], # 2
    [1.0, 0.0], # 2
    [0.0, 0.0], # 3
    [0.0, 1.0]  # 0
], dtype=np.float32)

Then, we add the `StaticMeshComponent` specifying a name, we chose "triangle" in this case.

Then, we specify in an array all the vertex data, in our case we have the vertices and texture coordinates that we created above.

In [10]:
scene.add_component(quad, StaticMeshComponent('quad', [vertices, texture_coords]))

The last component that we need in our example is the `MaterialComponent`.

We just have to give the name of the material we have built before, i.e.: "M_Unlit".

In [11]:
scene.add_component(quad, MaterialComponent('M_UnlitTextured'))

Now that we have the components added to the entities, we have to register systems that we are gonna use to the scene.
In this example we are using two systems:
- `TransformSystem`
- `OpenGLStaticMeshRenderingSystem`

When creating a system we have to provide to the constructor an array that holds the component(s) that the system operates on.

Then the system uses these component(s) to filter all the entities that have them.
- For example, the `TransformSystem` operates on all the entities that have a `TransformComponent`, similarly the `OpenGLStaticMeshRenderingSystem` operates on all the entities that have a `StaticMeshComponent`, a `MaterialComponent` and a `TransformComponent`.

We can register them to the scene by calling the `register_system` method.

In [12]:
scene.register_system(TransformSystem([TransformComponent]))
scene.register_system(OpenGLStaticMeshRenderingSystem([StaticMeshComponent, MaterialComponent, TransformComponent]))

Now that our scene has all the entites, components and systems added we can add it to the `SceneManager` by calling the `add_scene` method.

In [13]:
SceneManager().add_scene(scene)

Finally, we can start the `Application` by calling the `start` method.

In [14]:
Application().start()